In [ ]:
import nglview 
import ipywidgets
import pytraj as pt
import os


#pdb_code = "6AOG" # well structured protein 


In [ ]:
#load .pdb files from your dir 
pro_a = nglview.show_file("/protein_set/pf3d7_0827400_Q8IBA6.pdb")
pro_b = nglview.show_file("/protein_set/pf3d7_0904300-C0H518.pdb")
pro_c = nglview.show_file("/protein_set/pf3d7_0925100-Q8I2Q7.pdb")
pro_d = nglview.show_file("/protein_set/pf3d7_1109100-Q8IIR3.pdb")
pro_e = nglview.show_file("/protein_set/pf3d7_1316700-A0A5K1K8Q0.pdb")
pro_f = nglview.show_file("/protein_set/pf3d7_0408800-Q9U0J8.pdb")
pro_g = nglview.show_file("/protein_set/pf3d7_0419400-Q8I1P3.pdb")
pro_h = nglview.show_file("/protein_set/pf3d7_0929300-Q8I2L7.pdb")
pro_i = nglview.show_file("/protein_set/pf3d7_1024000-Q8IJG5.pdb")
pro_j = nglview.show_file("/protein_set/pf3d7_1032300-Q8IJ84.pdb")
pro_k = nglview.show_file("/protein_set/pf3d7_1106500-Q8IIT8.pdb")
pro_l = nglview.show_file("/protein_set/pf3d7_1117200_A0A144A0B3.pdb")
pro_m = nglview.show_file("/protein_set/pf3d7_1136200-Q8II02.pdb")
pro_n = nglview.show_file("/protein_set/pf3d7_1227300-Q8I5E5.pdb")
pro_o = nglview.show_file("/protein_set/pf3d7_1306200-Q8IEQ3.pdb")

In [ ]:
view = nglview.show_structure_file(pro_o, default=False)

view.add_representation(repr_type='cartoon', 
                        selection='not het',
                        colorScheme = 'atomindex')

#view.add_representation(repr_type='surface', 
#                       selection= 'not het', 
#                       colorScheme = '#cccccc',
#                        lowResolution= False,
#                        opacity=0,
                        # 0: low resolution 
#                        smooth=1 )

view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])

view



In [ ]:
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view.render_image()
view._display_image()

In [ ]:
view = nglview.show_structure_file(pro_a, default=False)
view.add_representation(repr_type='cartoon', 
                        selection='not het',
                       colorScheme = 'atomindex')
view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])

view

In [ ]:
#3 finding pockets 

import fpocket_run

fpocket_all_pockets = "fpocket_all_pockets.zip"
fpocket_summary = "fpocket_summary.json"
prop = {
    "min_radius": 3,
    "max_radius": 6,
    "num_spheres": 35
}

fpocket_run(input_pdb_path=pro_a,
        output_pockets_zip = fpocket_all_pockets,
        output_summary=fpocket_summary,
        properties=prop)

In [ ]:
# checking pockets score , drugabbility 
import json

with open(fpocket_summary, 'r') as json_file:
    data = json.load(json_file)
    print(json.dumps(data, indent=4))

In [ ]:
#filtering cavities 
import fpocket_filter

fpocket_filter_pockets = "fpocket_filter_pockets.zip"
prop = {
    "volume": [800, 2000]
}

fpocket_filter(input_pockets_zip=fpocket_all_pockets,
                input_summary = fpocket_summary,
                output_filter_pockets_zip=fpocket_filter_pockets,
                properties=prop)

In [ ]:
# extract filtered pockets 
import os
import shutil

from pathlib import Path, PurePath
import zipfile

#if Path(pockets_dir).exists(): shutil.rmtree(pockets_dir) 
if os.path.exists("./pockets"):
    shutil.rmtree("./pockets")
os.makedirs("./pockets")
#os.mkdir(pockets_dir)

with zipfile.ZipFile(fpocket_filter_pockets, 'r') as zip_ref:
    zip_ref.extractall("./pockets")

path_pockets = [str(i) for i in Path("./pockets").iterdir()]
path_pockets_pdb = [str(i) for i in Path("./pockets").iterdir() if PurePath(i).suffix == '.pdb']
path_pockets_pqr = [str(i) for i in Path("./pockets").iterdir() if PurePath(i).suffix == '.pqr']

In [ ]:
#visulaization 
import re
import random

# colors
r = lambda: random.randint(0,255)

# load structure
view = nglview.NGLWidget()
c = view.add_component(pro_a)

# load cavities, pockets and create pocketNames 
c = {}
p = {}
pocketNames = []
for pock in path_pockets:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    suff = g[0][1]
    if not [item for item in pocketNames if ('pocket' + i) in item]: pocketNames.append(('pocket' + i, int(i)))

    if suff == 'pdb':
        c[i] = view.add_component(pock)
        c[i].clear()
    else:
        p[i] = view.add_component(pock)
        p[i].clear()

# sort 
pocketNames.sort(key=lambda tup: tup[1])
        
#  cavities
for pock in path_pockets_pdb:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    c[i].add_surface(color='#%02X%02X%02X' % (r(),r(),r()),  
                     radius='1.5',
                     lowResolution= True,
                     # 0: low resolution 
                     smooth=1,
                     useWorker= True,
                     wrap= True)
    
# representation
for pock in path_pockets_pqr:
    g = re.findall('(?:pocket)(\d+)(?:_\w+)\.(\w+)', pock)
    i = g[0][0]
    p[i].add_surface( component=i, color='skyblue', surfaceType= 'av', contour=True )

view.center()
view._remote_call('setSize', target='Widget', args=['','600px'])
view

#labels
code = """
var stage = this.stage;
var view = this.stage.viewer;
var clist_len = stage.compList.length;
var i = 0;
for(i = 0; i <= clist_len; i++){
    if(stage.compList[i] != undefined && stage.compList[i].structure != undefined && stage.compList[i].object.name.indexOf('pqr') != -1) {        

        var elm = document.createElement("div");
        elm.innerText = 'pocket' + stage.compList[i].object.name.match(/\d+/g)
        elm.style.color = "black";
        elm.style.background = "rgba(201, 149, 6, .8)";
        elm.style.padding = "8px";
        
        stage.compList[i].addAnnotation(stage.compList[i].structure.center, elm)
    }
}
"""

view._execute_js_code(code)

view

In [ ]:
pocketNames